# OpenSSL

OpenSSL [https://openssl.org]() 

> OpenSSL is a robust, commercial-grade, and full-featured toolkit for the Transport Layer Security (TLS) and Secure Sockets Layer (SSL) protocols. It is also a general-purpose cryptography library.

Those interested in the history of OpenSSL can find an interesting blog post here [https://www.openssl.org/blog/blog/2018/12/20/20years/]()

## Common usages

Some common usages of openssl in the pki system are related to generating new certificate, certificate requests, manually verifying certificates, and inspecting them. These commands are generally the same in all variants of linux, however, some implementations may not have every option compiled into the package (an example would be the rehash function is not available in Red Hat). 

For this section we will just be looking at openssl from the client perspective, as it relates to the PKI system. 

Taking a look at the package.

```
# rpm version
rpm -q openssl --info
```

In [ ]:
%%bash
dpkg-query -s openssl 

One of the most common uses for `openssl` is inspection of X.509 certificates, which we touch briefly on it in the system certificate section to look at a certificate siganture. 

Lets take a look at the Let's Encrypt root certificate.

In [ ]:
%%bash
openssl x509 -in /etc/ssl/certs/DST_Root_CA_X3.pem -noout -text

For a client certificate, I am going to grab the ohiolinux.org certificate from the website.

In [ ]:
%%bash
openssl s_client -4 -no_ign_eof ohiolinux.org:443 <<< "Q" > tmp
# I want to show the extra data that comes back with the s_client
echo s_client output; echo; cat tmp; echo
# and show a cool function of openssl, used to clean up a certificate file
openssl x509 -in tmp -out certificates/ohiolinux.crt
echo clean cert; echo; cat certificates/ohiolinux.crt; echo
rm -f tmp

Now that we have a client certificate, we can start to inspect it. 

First we can inspect the valid dates.

In [ ]:
%%bash 
openssl x509 -in certificates/ohiolinux.crt -noout -startdate -enddate

In [ ]:
%%bash
# checking that it will be valid in the next second
openssl x509 -in certificates/ohiolinux.crt -noout -checkend 1

In [ ]:
%%bash
# checkend is in seconds, bash calc to look at the cert 120 days from now $((sec * minutes * hours * days))
openssl x509 -in certificates/ohiolinux.crt -noout -checkend $((60 * 60 * 24 * 120))

While the certificate may be valid locally, we would have to confirm that it has not been revoked. To do this we can check the certificate for that information. 

In [ ]:
%%bash
openssl x509 -in certificates/ohiolinux.crt -noout -ocsp_uri

In [ ]:
%%bash
openssl ocsp -no_nonce -noverify -issuer /etc/ssl/certs/DST_Root_CA_X3.pem -cert certificates/ohiolinux.crt -url http://ocsp.int-x3.letsencrypt.org -header Host=ocsp.in-x3.letsencrypt.org -text

The reason we get this error is that we do not have the chain certificate. The root certificate is not the issuer. 

Lets grab the correct issuer, this is offered by the server if we request it.

In [ ]:
%%bash
openssl s_client -4 -showcerts -no_ign_eof ohiolinux.org:443 <<< "Q" > tmp
# we need to pull out the cert that is not the current cert we have
diff --left-column tmp certificates/ohiolinux.crt | sed -e "s/^<\ //" > tmp2
echo chain_cert; echo; cat tmp2; echo
# and clean that up
openssl x509 -in tmp2 -out certificates/olchain.crt
echo clean_cert; echo; cat certificates/olchain.crt; echo
# clean up the tmp files
rm -f tmp tmp2

Now lets try the OCSP request again.

In [ ]:
%%bash
openssl ocsp -no_nonce -noverify -issuer certificates/olchain.crt -cert certificates/ohiolinux.crt -url http://ocsp.int-x3.letsencrypt.org -header Host=ocsp.int-x3.letsencrypt.org -text

## Some closing remarks

In this section we looked at some very generic uses of openssl when it comes to PKI. We will be looking much more in depth to openssl when we start to talk about the construction of a certificate authority. 

The openssl tool has a great deal more components, but without access to some of the certificate authority structure we will not be able to effectively demostrate it. I will cover some additional concepts a bit later.